In [1]:
import requests
from xml.etree import ElementTree
import os
import sys
import numpy as np
import pandas as pd
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from surprise import Reader, Dataset, SVD, dump
from surprise.model_selection import cross_validate  # thay cho evaluate
import surprise.accuracy as accuracy                 # giữ accuracy
from collections import defaultdict

# Custom libraries
sys.path.append('../Util')
from loader import get_books, get_book_dataframe, get_book_features, get_mapper
from joiner import get_ratings, get_joint
from reduction import reduce_matrix, get_sparse
import secret # need to make this and add goodreads_api key

ModuleNotFoundError: No module named 'secret'

In [16]:
def get_id_from_username(username, api_key):
    response = requests.get('https://www.goodreads.com/user/show/?key='+api_key+'&username='+username+'&format=xml')
    tree = ElementTree.fromstring(response.content)
    try:
        user_id = tree.find('user').find('id').text
        return user_id
    except:
        raise ValueError('Invalid Goodreads username, not id returned')
        return None

In [ ]:
# Set this to where you save and load all data
data_path = '../goodbooks-10k/'

In [13]:
# Get dataframe from books
books = get_book_dataframe(data_path)

found books_dataframe in file...


In [14]:
mapper = get_mapper(data_path + 'books.csv')

In [15]:
# make an array for myself
q = np.zeros((10000), dtype = np.int)

In [20]:
username = secret.USERNAME
api_key = secret.API_KEY

AttributeError: module 'secret' has no attribute 'USERNAME'

In [18]:
user_id = get_id_from_username(username, api_key)
user_id

'26809953'

In [18]:
page = 1
while True:
    response = requests.get('https://www.goodreads.com/review/list/?v=2&id='+user_id+'&shelf=read&format=xml&key='+api_key+'&per_page=200&page=' + str(page))
    tree = ElementTree.fromstring(response.content)
    reviews = tree.find('reviews')
    for review in reviews:
        goodreads_book_id = str(review.find('book').find('id').text)
        if goodreads_book_id in mapper:
            book_id = int(mapper[goodreads_book_id])
            rating = int(review.find('rating').text)
            q[book_id-1] = float(rating)
    page += 1
    
    print(len(reviews))
    if len(reviews) < 1:
        break

200
62
0


In [120]:
np.save('../.tmp/user_vector', q)

In [121]:
for i in range(len(q)):
    if q[i] != 0:
        title = books.iloc[i]['title']
        print("%s --> %s" % (q[i], title))

5 --> The Hunger Games (The Hunger Games, #1)
5 --> Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
4 --> To Kill a Mockingbird
4 --> The Great Gatsby
4 --> The Fault in Our Stars
4 --> The Hobbit
5 --> The Catcher in the Rye
3 --> Pride and Prejudice
5 --> The Kite Runner
4 --> Divergent (Divergent, #1)
3 --> 1984
4 --> Animal Farm
3 --> The Diary of a Young Girl
4 --> The Girl with the Dragon Tattoo (Millennium, #1)
4 --> Catching Fire (The Hunger Games, #2)
3 --> Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)
5 --> The Fellowship of the Ring (The Lord of the Rings, #1)
3 --> Mockingjay (The Hunger Games, #3)
3 --> Harry Potter and the Order of the Phoenix (Harry Potter, #5)
4 --> Harry Potter and the Chamber of Secrets (Harry Potter, #2)
3 --> Harry Potter and the Goblet of Fire (Harry Potter, #4)
4 --> Harry Potter and the Deathly Hallows (Harry Potter, #7)
3 --> Harry Potter and the Half-Blood Prince (Harry Potter, #6)
3 --> Lord of the Flies
4 --> Romeo and 

Đoạn code thay thế do không thể lấy dữ liệu từ api

In [2]:
import pandas as pd
import numpy as np


In [3]:
# Đọc dữ liệu từ ratings.csv
ratings = pd.read_csv('../goodbooks-10k/ratings.csv')


In [5]:
# Khởi tạo vector người dùng
q = np.zeros((10000), dtype=float)

In [6]:
# Chọn user_id cụ thể
user_id = 1  # Thay bằng user_id bạn muốn tạo vector

In [7]:
# Ánh xạ đánh giá vào vector
for _, row in ratings[ratings['user_id'] == user_id].iterrows():
    book_id = row['book_id']
    rating = row['rating']
    q[book_id - 1] = rating

In [8]:
# Lưu vector người dùng
np.save('../.tmp/user_vector', q)

In [9]:
# In các sách mà người dùng đã đánh giá
books = pd.read_csv('../goodbooks-10k/books.csv')
for i in range(len(q)):
    if q[i] != 0:
        title = books.iloc[i]['title']
        print(f"{q[i]} --> {title}")

5.0 --> To Kill a Mockingbird
4.0 --> Pride and Prejudice
5.0 --> The Kite Runner
4.0 --> 1984
3.0 --> The Girl with the Dragon Tattoo (Millennium, #1)
3.0 --> The Lovely Bones
4.0 --> The Help
4.0 --> Of Mice and Men
4.0 --> Memoirs of a Geisha
5.0 --> The Alchemist
4.0 --> The Giver (The Giver, #1)
2.0 --> The Time Traveler's Wife
2.0 --> Eat, Pray, Love
3.0 --> Little Women (Little Women, #1)
4.0 --> Jane Eyre
5.0 --> Life of Pi
4.0 --> Water for Elephants
3.0 --> The Book Thief
3.0 --> The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)
3.0 --> The Secret Life of Bees
3.0 --> The Curious Incident of the Dog in the Night-Time
4.0 --> Gone with the Wind
3.0 --> A Thousand Splendid Suns
5.0 --> Ender's Game (Ender's Saga, #1)
3.0 --> The Shining (The Shining #1)
3.0 --> Sense and Sensibility
5.0 --> The Glass Castle
3.0 --> The Giving Tree
1.0 --> One Hundred Years of Solitude
4.0 --> The Picture of Dorian Gray
3.0 --> The Girl Who Played with Fire (Millennium,